# GEN level distributions

In [ ]:
%load_ext autoreload

In [2]:
%autoreload 2

In [1]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#test = "/vols/cms/akd116/Offline/output/SM/2019/Mar4_2016_gen/GluGluToHToTauTau_M-125-nospinner_mt_2016.root"
test = "/vols/build/cms/akd116/newest/run/CMSSW_8_0_25/src/UserCode/ICHiggsTauTau/Analysis/HiggsTauTauRun2/output/VBFHToTauTauUncorrelatedDecay_Filtered_2018_tt_0.root"
#test = "/vols/cms/akd116/Offline/output/SM/2020/Apr26_2018_gen_cpdecays/VBFHToTauTauUncorrelatedDecay_Filtered_tt_2018.root"

f = uproot.open(test)["gen_ntuple"]
f.keys()
df = f.pandas.df([
    "cp_channel",
    "aco_angle_1",
    "aco_angle_2",
    "aco_angle_3",
    "aco_angle_4",
    "wt_cp_sm",
    "wt_cp_ps",
    "wt_cp_mm",
    "count_taus",
],namedecode="utf-8")

#df = pd.DataFrame()
#for file_number in np.linspace(0, 1, 1):
#    #test = f"/vols/cms/akd116/Offline/output/SM/2020/Apr26_2018_gen_cpdecays/VBFHToTauTauUncorrelatedDecay_Filtered_2018_tt_{int(file_number)}.root"
#    f = uproot.open(test)["gen_ntuple"]
#    f.keys()
#    df = pd.concat([df, f.pandas.df([
#        "cp_channel",
#        "aco_angle_1",
#        "aco_angle_2",
#        "aco_angle_3",
#        "aco_angle_4",
#        "wt_cp_sm",
#        "wt_cp_ps",
#        "wt_cp_mm",
#        "genpt_2",
#        "cp_sign_1",
#    ],namedecode="utf-8")])
#df.set_index(["cp_channel"], inplace=True)

In [4]:
# options
import dftools
from plotting import var_kw
import itertools
cp_channel = 2
plot_var = "aco_angle_1"
weights = ["wt_cp_sm", "wt_cp_ps",]# "wt_cp_mm"]
legend_items = [
    r"$\mathrm{SM}\ (\phi_{\tau}=0)$",
    r"$\mathrm{PS}\ (\phi_{\tau}=\pi/2)$",
    r"$\mathrm{MM}\ (\phi_{\tau}=\pi/4)$",
]

In [5]:
plt.style.use('cms')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = itertools.cycle(prop_cycle.by_key()['color'])

fig, ax = plt.subplots(figsize=(2.9, 2.6), dpi=200,)

dftools.draw.cms_label(ax, "Simulation", lumi=59.7)

bin_edges = np.linspace(0, 2.*np.pi, endpoint=True, num=51)
bin_low_edge = bin_edges[:-1]
for weight in weights:
    sum_w, _ = np.histogram(
        df.loc[df.eval(f"cp_channel=={cp_channel}"), plot_var],
        bins=bin_edges,
        weights=df.loc[df.eval(f"cp_channel=={cp_channel}"), weight],
    )
    sum_w_total = sum_w.sum()
    sum_w /= sum_w_total
    sum_ww, _ = np.histogram(
        df.loc[df.eval(f"cp_channel=={cp_channel}"), plot_var],
        bins=bin_edges,
        weights=(df.loc[df.eval(f"cp_channel=={cp_channel}"), weight])**2,
    )
    sum_ww /= (sum_w_total)**2
    color=next(colors),
    ax.hist(
        bin_low_edge,
        bins=bin_edges,
        histtype='step',
        weights=sum_w,
        color=color,
    )
    up = sum_w + np.sqrt(sum_ww)
    down = sum_w - np.sqrt(sum_ww)
    ax.fill_between(
        bin_edges,
        list(up)+[up[-1]],
        list(down)+[down[-1]],
        step='post',
        alpha=0.2,
        color=color,
    )
    
ax.legend(
    legend_items,
    loc=3, 
    labelspacing=0.2, 
    fontsize=9.,
    framealpha=0.8,
)

ax.set_xlim(0, 2.*np.pi)
ax.set_ylabel("Normalised events")
try:
    ax.set_xlabel(var_kw[plot_var])
except KeyError:
    print(f"{plot_var} not defined in var_kw")
    ax.set_xlabel(plot_var.replace("_"," "))
ax.set_xticks([0, np.pi/2, np.pi, 1.5*np.pi, 2.*np.pi])
ax.set_xticklabels([0, r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])

fig.savefig(f"plots/vbf_gen_{plot_var}_cp{cp_channel}.pdf", bbox_inches='tight')

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
